In [30]:
! pip install langchain faiss-cpu sentence-transformers openai tiktoken rouge-score nltk  python-dotenv langchain-community langchain_openai rouge nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from pathlib import Path
from langchain.schema import Document
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
import re

def load_documents_from_folder(folder_path):
    """Load each .txt file in the folder as a separate Document."""
    txt_files = Path(folder_path).glob("*.txt")
    documents = []
    
    for file in txt_files:
        text = file.read_text(encoding="utf-8")
        clean_text = re.sub(r'\s+', ' ', text.strip())  # Clean and normalize
        doc = Document(page_content=clean_text, metadata={"source": file.name})
        documents.append(doc)
    
    return documents

# Load documents from folder
folder_path = "Dataset/ancient_greece_data"
documents = load_documents_from_folder(folder_path)

# Create embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Build FAISS index
vectorstore = FAISS.from_documents(documents, embedding_model)

# SAVE TO LOCAL STORE
vectorstore.save_local("faiss_index_ancient_greece")

/var/folders/v8/drg1lhv96m5dw884d9q2_y9c0000gn/T/ipykernel_8500/2977521100.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/nishtha/Documents/FAB-RAG-Assignment/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# LOAD WHEN LOADING FROM LOCAL
vectorstore = FAISS.load_local("faiss_index_ancient_greece", embedding_model, allow_dangerous_deserialization=True)

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
# vectorstore.similarity_search_with_relevance_scores(query="who was socrates", k=4)

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


contextualize_q_prompt =  """Given above  chat history and the below  latest user question
    which might reference context in the chat history,
    formulate a standalone question which can be understood
    without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is.
    Below is the latest  question:

    {input}
    """


contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that rewrites follow-up questions into standalone questions using chat history."),
        MessagesPlaceholder("chat_history"),
        ("human", """Given the above chat history and the latest user question below,
reformulate it into a standalone question. Do not answer the question.
If it's already standalone, return it as is.

Latest user question:
{input}"""),
    ]
)


history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", (
                        """You are an assistant for question-answering tasks.
                        "Answer this question using the provided context only.
                        If you dont know the answer, just say 'I dont know'
                        {context}"""
                    )),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)

contextual_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    contextual_rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [15]:
conversational_rag_chain

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x3036c82c0>, input_messages_key='input', output_messages_key='answer', history_messages_key='chat_history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [16]:
response=conversational_rag_chain.invoke(
    {"input": "who is socrates"},
    config={
        "configurable": {"session_id": "abc1235"}
    },  # constructs a key "abc123" in `store`.
)
print(response["answer"])

Socrates was a philosopher who lived in Athens during the 5th century BCE and is considered the father of Western philosophy. He is famous for his method of inquiry known as the Socratic Method, which involved questioning others to stimulate critical thinking and the search for truth. Although Socrates left no written works behind, his ideas and teachings were passed down through his disciple, Plato. Socrates believed that wisdom came from acknowledging one's own ignorance and engaging in open dialogue with others. His commitment to truth and ethical conduct ultimately led to his trial and execution.


In [17]:
response=conversational_rag_chain.invoke(
    {"input": "where did he lived"},
    config={
        "configurable": {"session_id": "abc1235"}
    },  # constructs a key "abc123" in `store`.
)
print(response["answer"])

Socrates lived in Athens.
